In [1]:
using LinearAlgebra

In [2]:
A = zeros(Float64, 100, 3)

100×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [3]:
using DataFrames
using CSV

In [7]:
save_data1 = DataFrame(A, :auto)
CSV.write("./test.csv", save_data1)

"./test.csv"

In [16]:
B = [3.0, 3.0, 5.0, 6.0]

4-element Vector{Float64}:
 3.0
 3.0
 5.0
 6.0

In [19]:
C = [A, B]
D = vcat(C...)

6-element Vector{Float64}:
 2.0
 1.0
 3.0
 3.0
 5.0
 6.0

In [1]:
using Flux

In [2]:
model = Chain(Dense(zeros(Float64, 4,2), zeros(Float64,4), relu), Dense(zeros(Float64, 3,4), zeros(Float64,3)))

Chain(
  Dense(2 => 4, relu),                  # 12 parameters  (all zero)
  Dense(4 => 3),                        # 15 parameters  (all zero)
)                   # Total: 4 arrays, 27 parameters, 472 bytes.

In [3]:
model([1.0,2.0])

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [2]:
mutable struct models
    model
    opt
    loss
end

In [27]:

model = Chain(Dense(2, 4, relu), Dense(4, 4, relu), Dense(4, 3))
opt = ADAM()

ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [22]:
function loss_F(model1, x, t, id)
    y = model1(x)
    l = (y' * y + t)^id
end

loss_F (generic function with 1 method)

In [5]:
typeof(model)

Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}

In [23]:
pp = Flux.params(model)

Params([Float32[-0.6409421 -0.48671305; -0.049359918 -0.953117; -0.62846947 0.072873235; -0.48038542 0.16628885], Float32[0.0, 0.0, 0.0, 0.0], Float32[0.30316955 0.32862136 0.68905085 0.034558468; 0.71608937 -0.8556327 -0.76498353 0.28445715; -0.38000605 0.5602685 0.4591866 0.25020695], Float32[0.0, 0.0, 0.0]])

In [28]:
grads = Flux.gradient(Flux.params(model)) do
    loss_F(model,[1.0, 2.0], 1.0, 2)
end

Grads(...)

In [29]:
Flux.Optimise.update!(opt, Flux.params(model), grads)

In [30]:
Flux.params(model)

Params([Float32[-0.7304796 -0.6669884; 0.053862214 -0.9574983; 0.63835186 0.78019816; -0.9971867 0.6426752], Float32[0.0, 0.0, -0.0009999999, 0.0009999999], Float32[0.71975607 -0.33459142 -0.64171374 0.6262861; -0.7356834 0.43697602 0.38250968 -0.051746085; 0.027988514 0.25214875 0.76845664 -0.7908854; -0.8467408 0.38830227 0.4709868 0.2552434], Float32[0.0, 0.0009999999, -0.001, -0.0009999999], Float32[-0.04206128 0.5674432 -0.8883245 0.17535059; -0.5378207 0.5133928 -0.6044832 -0.47304583; -0.5062089 -0.7473814 0.5360178 0.15815876], Float32[0.001, 0.001, -0.0009999999]])

In [4]:
struct netQAgt
    n_act::Int
    input_size::Int
    n_dense::Int
    ϵ::Float64
    γ::Float64
    
    #filepath::String
end

function init_nQ()
    n_act::Int = 3
    input_size::Int = 4
    n_dense::Int = 32
    ϵ::Float64 = 0.1
    γ::Float64 = 0.9
    filepath::String = "test"

    return n_act, input_size, n_dense, ϵ, γ, filepath
end

init_nQ (generic function with 1 method)

In [13]:
function build_model(nq::netQAgt)
    model = Chain(Dense(nq.input_size, nq.n_dense, relu), Dense(nq.n_dense, nq.n_act))
    opt = ADAM()
    loss(x,y) = Flux.mse(model(x),y)

    return model, opt, loss
end


build_model (generic function with 1 method)

In [14]:
nq = netQAgt(init_nQ()...)

netQAgt(3, 4, 32, 0.1, 0.9, "test")

In [15]:
m_test = models(build_model(nq)...)

models(Chain(Dense(4 => 32, relu), Dense(32 => 3)), ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), var"#loss#3"{Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}}(Chain(Dense(4 => 32, relu), Dense(32 => 3))))

In [16]:
b = m_test.model(ones(4))

3-element Vector{Float64}:
  0.07427979263073353
  0.12010678845424419
 -0.4972820025765046

In [17]:
c = m_test.loss(ones(4),ones(3))

1.2910077870985366

In [18]:
println(m_test.model)

Chain(Dense(4 => 32, relu), Dense(32 => 3))


In [21]:
using BSON

In [23]:
bson("test.bson", m_test.model)

LoadError: MethodError: no method matching bson(::String, ::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}})
[0mClosest candidates are:
[0m  bson(::String; kws...) at ~/.julia/packages/BSON/rOaki/src/write.jl:89
[0m  bson(::String, [91m::AbstractDict[39m) at ~/.julia/packages/BSON/rOaki/src/write.jl:87

In [2]:
model = Chain(Dense(2, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

Chain(
  Dense(2 => 5, relu),                  # 15 parameters
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 27 parameters, 364 bytes.

In [3]:
function loss(x, y)
    return (x' * x - y^2)
end

loss (generic function with 1 method)

In [4]:
opt = ADAM()
data = [([1.0, 2.0], 0.5)]

1-element Vector{Tuple{Vector{Float64}, Float64}}:
 ([1.0, 2.0], 0.5)

In [5]:
Flux.train!(loss,Flux.params(model), data, opt)

In [26]:
using BSON: @save

In [37]:
typeof(model)

Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}}

In [27]:
@save "test.bson" model

In [28]:
m_test.model = Chain(Dense(10, 5, NNlib.relu), Dense(5, 2), NNlib.softmax)

Chain(
  Dense(10 => 5, relu),                 # 55 parameters
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 67 parameters, 524 bytes.

In [38]:
m_test.model[1]

Dense(10 => 5, relu)  # 55 parameters

In [40]:
@save "test.bson" m_test.model

LoadError: LoadError: Unrecognised @save expression m_test.model
in expression starting at In[40]:1

In [44]:

bson("test2.bson", model)

LoadError: MethodError: no method matching bson(::String, ::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}})
[0mClosest candidates are:
[0m  bson(::String; kws...) at ~/.julia/packages/BSON/rOaki/src/write.jl:89
[0m  bson(::String, [91m::AbstractDict[39m) at ~/.julia/packages/BSON/rOaki/src/write.jl:87

In [49]:
@save "test2.bson" model

In [52]:
test = m_test.model |> cpu

Chain(
  Dense(10 => 5, relu),                 # 55 parameters
  Dense(5 => 2),                        # 12 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 67 parameters, 524 bytes.

In [53]:
@save "test2.bson" test

In [54]:
using BSON: @load

In [59]:
m_test2 = models(build_model(nq)...)


models(Chain(Dense(4 => 32, relu), Dense(32 => 3)), ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), var"#loss#3"{Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}}(Chain(Dense(4 => 32, relu), Dense(32 => 3))))

In [69]:
@load "test2.bson" test

In [70]:
q = [1.0 2.0 3.0 5.0]

1×4 Matrix{Float64}:
 1.0  2.0  3.0  5.0

In [6]:
mutable struct models
    Kt
    model
    opt
    loss
end

In [24]:
function build_model()
    Kt = ones(Float64,2)
    model = Chain(Dense(2, 4, relu), Dense(4, 2))
    opt = ADAM()
    loss(v, x, y) = (model(x)' * model(x) - y^2 + sum(v))

    return Kt, model, opt, loss
end

build_model (generic function with 1 method)

In [25]:
m = models(build_model()...)

models([1.0, 1.0], Chain(Dense(2 => 4, relu), Dense(4 => 2)), ADAM(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), var"#loss#6"{Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}}(Chain(Dense(2 => 4, relu), Dense(4 => 2))))

In [29]:
Loss(x,y) = m.loss(m.Kt, x, y)
Flux.train!(Loss,Flux.params(m.model), data, m.opt)

In [15]:
Flux.params(m.model)

Params([Float32[0.010531068 -0.35521567; -0.91696393 -0.100117445; -0.2657094 -0.6087868; -0.7047603 0.23531306], Float32[0.0, 0.0, 0.0, 0.0], Float32[0.70487535 0.73120105 -0.19861972 0.3961799; -0.67083395 0.97618544 -0.87142813 0.42663658], Float32[0.0, 0.0]])

In [30]:
Loss([1.0,2.0], 0.5)

2.1899924207384402

In [31]:
m.Kt = [2.0, 1.0]

2-element Vector{Float64}:
 2.0
 1.0

In [32]:
Loss([1.0,2.0], 0.5)

3.1899924207384402

In [35]:
using LinearAlgebra

In [75]:
function vec_to_matrix(v::Vector{Float64})
    d::Int = sqrt(length(v))
    M = zeros(ComplexF64,d,d)
    for i in 1:d
        for j in i:d
            #l = (i-1)*d + 2*(j-1)
            if(i==j)
                l = 2(d+1-i)*(i-1) + (i-1)^2 + 1
                M[i,j] = v[l]
            else
                l = 2(d+1-i)*(i-1) + (i-1)^2 + 2*(j-i)
                M[i,j] = v[l] + im*v[l+1]
            end
        end
    end
    Kt = Hermitian(M)
    
    return Kt
end

function matrix_to_vec(M::Hermitian{ComplexF64, Matrix{ComplexF64}})
    X =UpperTriangular(M)
    B=filter(x->x!=0.00,imag.(X[:]))
    append!(B,filter(x->x!=0.0,real.(X[:])))
    #d = size(M)[1]
    #v::Vector{Float64} = []
    return B
end


matrix_to_vec (generic function with 2 methods)

In [70]:
A = [2.0 1.0+1.0im; 1.0-1.0im 2.0]

2×2 Matrix{ComplexF64}:
 2.0+0.0im  1.0+1.0im
 1.0-1.0im  2.0+0.0im

In [73]:
A = Hermitian(A)
typeof(A)

Hermitian{ComplexF64, Matrix{ComplexF64}}

In [76]:
V = matrix_to_vec(A)

4-element Vector{Float64}:
 1.0
 2.0
 1.0
 2.0

In [39]:
e, v = eigen(B)

Eigen{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 0.5857864376269043
 3.414213562373095
vectors:
2×2 Matrix{ComplexF64}:
       0.5+0.5im       0.5+0.5im
 -0.707107+0.0im  0.707107+0.0im

In [37]:
B = vec_to_matrix(A)

2×2 Hermitian{ComplexF64, Matrix{ComplexF64}}:
 2.0+0.0im  1.0+1.0im
 1.0-1.0im  2.0+0.0im

In [67]:
e,v = eigen(vec_to_matrix(A))
d = e' * e

11.999999999999998

In [41]:
C = matrix_to_vec(B)

4-element Vector{Float64}:
 2.0
 1.0
 1.0
 2.0

In [43]:
D = A + 0.2/10*C

4-element Vector{Float64}:
 2.04
 1.02
 1.02
 2.04

In [44]:
S = A' * A

10.0

In [45]:
D = A-C

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [46]:
B[1,:]

2-element Vector{ComplexF64}:
 2.0 + 0.0im
 1.0 + 1.0im

In [47]:
D = 0.5*B[1,:]

2-element Vector{ComplexF64}:
 1.0 + 0.0im
 0.5 + 0.5im

In [52]:
function diff_norm(V::Vector{Float64})
    M = vec_to_matrix(V)
    e, v = eigen(M)
    n::Float64 = e' * e
    #n = sum(e[n]^2 for n in 1:size(e))
    return n
end

diff_norm (generic function with 1 method)

In [60]:
S = [1.0 2.0 2.0 1.0; 2.0 2.0 3.0 0.0]

2×4 Matrix{Float64}:
 1.0  2.0  2.0  1.0
 2.0  2.0  3.0  0.0

In [62]:
S2 = [1.0 2.0 2.0 1.0; 2.0 2.0 3.0 0.0]

2×4 Matrix{Float64}:
 1.0  2.0  2.0  1.0
 2.0  2.0  3.0  0.0

In [64]:
X = 0.5^5 * diff_norm(S[1,:]-S[2,:])

0.12500000000000003

In [65]:
S[1,:] += A

4-element Vector{Float64}:
 3.0
 3.0
 3.0
 3.0

In [88]:
A = [1.0 2.0; 2.0 1.0]

2×2 Matrix{Float64}:
 1.0  2.0
 2.0  1.0

In [89]:
A = UpperTriangular(A)

2×2 UpperTriangular{Float64, Matrix{Float64}}:
 1.0  2.0
  ⋅   1.0

In [90]:
At = zero(A)

2×2 UpperTriangular{Float64, Matrix{Float64}}:
 0.0  0.0
  ⋅   0.0

In [91]:
At[:] = real.(A)

2×2 UpperTriangular{Float64, Matrix{Float64}}:
 1.0  2.0
  ⋅   1.0

In [92]:
#using SparseArrays
Br = sparse(At)

2×2 SparseMatrixCSC{Float64, Int64} with 3 stored entries:
 1.0  2.0
  ⋅   1.0

In [93]:
At[:] = imag.(A)

2×2 UpperTriangular{Float64, Matrix{Float64}}:
 0.0  0.0
  ⋅   0.0

In [94]:
Bi = sparse(At)

2×2 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 

In [100]:
B = real.(Br.val)

ErrorException: type SparseMatrixCSC has no field val

In [98]:
Ar = real.(A)
Ai = imag.(A)

MethodError: MethodError: no method matching append!(::SparseMatrixCSC{Float64, Int64}, ::SparseMatrixCSC{Float64, Int64})
Closest candidates are:
  append!(!Matched::DataStructures.MutableLinkedList, ::Any...) at ~/.julia/packages/DataStructures/59MD0/src/mutable_list.jl:160
  append!(!Matched::Transducers.Transducer, ::Any, !Matched::Any) at ~/.julia/packages/Transducers/HBMTc/src/processes.jl:736
  append!(!Matched::BitVector, ::Any) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/bitarray.jl:782
  ...